In [2]:
# 검색을 받아 첫번째 url주소를 추출
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
from konlpy.tag import Hannanum

# from konlpy.tag import Okt
from gensim.models import Word2Vec
import pandas as pd


def get_first_search_result_url(query):
    # 웹드라이버 초기화
    driver = webdriver.Chrome()

    # Google 검색 페이지 열기
    driver.get(f"https://www.google.com/search?q={query}" + " 인재상")

    # 첫 번째 검색 결과 제목의 URL 가져오기
    try:
        first_title = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "h3.LC20lb.MBeuO.DKV0Md"))
        )
        url = first_title.find_element(By.XPATH, "./parent::a").get_attribute("href")
    except:
        url = None

    # 웹드라이버 종료
    driver.quit()
    return url


query = input("검색어를 입력하세요: ")
first_url = get_first_search_result_url(query)


# 명사만 추출하는 함수 정의
def extract_nouns_from_html(first_url):
    # 웹사이트의 HTML 코드 가져오기
    response = requests.get(first_url)
    # 인코딩 설정
    response.encoding = response.apparent_encoding
    html = response.text

    # BeautifulSoup을 사용하여 HTML 파싱
    soup = BeautifulSoup(html, "html.parser")

    # HTML에서 텍스트 추출
    text = soup.get_text()
    # Hannanum 형태소 분석기 초기화
    hannanum = Hannanum()
    # 명사만 추출하여 리스트에 저장
    nouns = hannanum.nouns(text)
    return nouns


# 명사만 추출
nouns = extract_nouns_from_html(first_url)

# # 중복제거 하지 않고 리스트로 출력하여 단어의 범위를 줄이지 않음.
# # 중복 제거를 위해 set으로 변환 후 출력
# unique_nouns = set(nouns)
# for noun in unique_nouns:
#     print(noun)
print("추출된 명사의 갯수: ", len(nouns))


# ==========================관련단어 코드
# Word2Vec 모델 로드
def load_word2vec_model(model_path):
    model = Word2Vec.load(model_path)
    return model


# 인재상에 해당하는 키워드 추출
def extract_related_keywords(nouns, model, target_words, threshold=0.6):
    related_keywords = []
    for word in target_words:
        for noun in nouns:
            try:
                similarity = model.wv.similarity(noun, word)
                if similarity >= threshold:
                    related_keywords.append(noun)
            except:
                pass
    return related_keywords


# Word2Vec 모델 로드
model_path = (
    "wiki.model"
)
model = load_word2vec_model(model_path)

# 인재상에 해당하는 키워드 리스트
target_words = ["인재", "소통", "능력", "책임", "성실"]

# 인재상에 해당하는 키워드 추출
related_keywords = extract_related_keywords(nouns, model, target_words)

unique_keywords = set(related_keywords)
print("인재상에 해당하는 키워드:", len(unique_keywords), "개")

df = pd.DataFrame({"인재상에 해당하는 키워드(중복제거)": list(unique_keywords)})
df

The chromedriver version (122.0.6261.128) detected in PATH at c:\Users\kosmo\Desktop\kosmo_baeksn\project3\resumate\data\chromedriver.exe might not be compatible with the detected chrome version (123.0.6312.59); currently, chromedriver 123.0.6312.58 is recommended for chrome 123.*, so it is advised to delete the driver in PATH and retry


추출된 명사의 갯수:  360
인재상에 해당하는 키워드: 8 개


,인재상에 해당하는 키워드(중복제거)
0,목표
1,책임
2,의지
3,겸손
4,실력
5,혁신
6,직무
7,전문성


In [1]:
from bs4 import BeautifulSoup
import requests
from konlpy.tag import Okt
from gensim.models import Word2Vec
import pandas as pd


def parse_html(url):
    response = requests.get(url)
    response.encoding = response.apparent_encoding
    html = response.text
    soup = BeautifulSoup(html, "html.parser")
    return soup


def extract_text_from_html(soup):
    text = soup.get_text()
    return text


def create_dataframe(data):
    df = pd.DataFrame(data, columns=["Text"])
    return df


def main():
    query = input("검색어를 입력하세요: ")
    url = f"https://www.google.com/search?q={query}" + " 인재상"

    soup = parse_html(url)
    text = extract_text_from_html(soup)

    okt = Okt()
    sentences = okt.morphs(text)

    model = Word2Vec.load("wiki.model")

    data = {"Text": sentences}
    df = create_dataframe(data)

    print("첫 번째 검색 결과 제목의 URL:", url)
    print("Word2Vec 모델 학습 완료")
    print("데이터 프레임 생성 완료")
    print(df)


if __name__ == "__main__":
    main()

AttributeError: 'Okt' object has no attribute 'sentences'